<a href="https://colab.research.google.com/github/adityabissa/AI/blob/main/manhattan8puzzleweek4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import itertools
import collections
import time

class Node:
    def __init__(self, puzzle, parent=None, action=None):
        self.puzzle = puzzle
        self.parent = parent
        self.action = action
        if self.parent is not None:
            self.g = parent.g + 1
        else:
            self.g = 0

    @property
    def score(self):
        return self.g + self.h

    @property
    def state(self):
        return str(self)

    @property
    def path(self):
        node, p = self, []
        while node:
            p.append(node)
            node = node.parent
        yield from reversed(p)

    @property
    def solved(self):
        return self.puzzle.solved

    @property
    def actions(self):
        return self.puzzle.actions

    @property
    def h(self):
        return self.puzzle.manhattan

    @property
    def f(self):
        return self.h + self.g

    def __str__(self):
        return str(self.puzzle)

class Solver:
    def __init__(self, start):
        self.start = start

    def solve(self):
        queue = collections.deque([Node(self.start)])
        seen = set()
        seen.add(queue[0].state)
        while queue:
            queue = collections.deque(sorted(list(queue), key=lambda node: node.f))
            node = queue.popleft()
            if node.solved:
                return node.path

            for move, action in node.actions:
                child = Node(move(), node, action)
                if child.state not in seen:
                    queue.appendleft(child)
                    seen.add(child.state)

class Puzzle:
    def __init__(self, board):
        self.width = len(board[0])
        self.board = board

    @property
    def solved(self):
        N = self.width * self.width
        return str(self) == ''.join(map(str, range(1, N))) + '0'

    @property
    def actions(self):
        def create_move(at, to):
            return lambda: self._move(at, to)

        moves = []
        for i, j in itertools.product(range(self.width), range(self.width)):
            direcs = {'R': (i, j - 1), 'L': (i, j + 1), 'D': (i - 1, j), 'U': (i + 1, j)}
            for action, (r, c) in direcs.items():
                if 0 <= r < self.width and 0 <= c < self.width and self.board[r][c] == 0:
                    move = create_move((i, j), (r, c)), action
                    moves.append(move)
        return moves

    @property
    def manhattan(self):
        distance = 0
        for i in range(self.width):
            for j in range(self.width):
                if self.board[i][j] != 0:
                    x, y = divmod(self.board[i][j] - 1, self.width)
                    distance += abs(x - i) + abs(y - j)
        return distance

    def shuffle(self):
        puzzle = self
        for _ in range(1000):
            puzzle = random.choice(puzzle.actions)[0]()
        return puzzle

    def copy(self):
        board = [row[:] for row in self.board]
        return Puzzle(board)

    def _move(self, at, to):
        copy = self.copy()
        i, j = at
        r, c = to
        copy.board[i][j], copy.board[r][c] = copy.board[r][c], copy.board[i][j]
        return copy

    def pprint(self):
        for row in self.board:
            print(row)
        print()

    def __str__(self):
        return ''.join(map(str, self))

    def __iter__(self):
        for row in self.board:
            yield from row

def get_user_input():
    print("Enter the initial state as a single line of numbers (0 for the empty space, e.g., '123456780'): ")
    initial_state_input = input().strip()
    initial_state = [int(num) for num in initial_state_input]

    print("Enter the goal state as a single line of numbers (0 for the empty space, e.g., '123456780'): ")
    goal_state_input = input().strip()
    goal_state = [int(num) for num in goal_state_input]

    width = int(len(initial_state) ** 0.5)
    initial_board = [initial_state[i * width:(i + 1) * width] for i in range(width)]
    goal_board = [goal_state[i * width:(i + 1) * width] for i in range(width)]

    return Puzzle(initial_board), Puzzle(goal_board)

if __name__ == "__main__":
    initial_puzzle, goal_puzzle = get_user_input()

    s = Solver(initial_puzzle)
    tic = time.time()
    p = s.solve()
    toc = time.time()

    steps = 0
    for node in p:
        print(node.action)
        node.puzzle.pprint()
        steps += 1

    print("Total number of steps: " + str(steps))
    print("Total amount of time in search: " + str(toc - tic) + " second(s)")


Enter the initial state as a single line of numbers (0 for the empty space, e.g., '123456780'): 
283164075
Enter the goal state as a single line of numbers (0 for the empty space, e.g., '123456780'): 
123804765
